## Model - YOLOv8
## type - one vs all  
## Dataset - VisDrone
## Test - Extended 

## only class 1 (padesterian) on multiple epoch

# 1. Install Dependencies

In [1]:
!pip install ultralytics
!pip install torch torchvision

import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.6/793.6 kB 14.1 MB/s eta 0:00:00


# 2. Create dataset function (Annotation)
as the format of annotation is as - object-class,x_center,y_center,width,height (all normalized as this is input format for YOLO5 prediction)

given annotation (in pixel) - bbox_left, bbox_top, bbox_width, bbox_height, score, object_category, truncation, occlusion

In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image



os.makedirs('/kaggle/working/yolo_annotations', exist_ok=True)

# Function to get image dimensions
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

# Function to convert annotations
def convert_to_yolo_format(filename, annotations, img_width, img_height, given_ ,recieved_path):
    output_annotations_path = recieved_path
    with open(os.path.join(output_annotations_path, filename), 'w') as out_file:
        for annotation in annotations:
            if filename=='242.txt':
                    annotation=[292,390,546,251,1,6,0,0]
            if filename=='1166.txt':
                    annotation=[895,280,200,485,1,6,1,0]
            if len(annotation) != 8:
                print(f"Error parsing annotation in {filename}: {annotation}")
                print(f"Ignoring malformed annotation: {annotation}")
                continue
            x_min, y_min, width, height, score, class_id, truncation, occlusion = map(float, annotation)
            if(class_id==0):
                continue
            # Convert to YOLO format
            center_x = x_min + (width / 2.0)
            center_y = y_min + (height / 2.0)
            center_x /= img_width
            center_y /= img_height
            width /= img_width
            height /= img_height
            class_id = class_id -1
            if(class_id == given_):
                class_id = 0
                out_file.write(f"{int(class_id)} {center_x} {center_y} {width} {height}\n")


In [3]:
def create_files_train_val(given_):
    # Process all annotation files    # Paths to the dataset
    annotations_path = '/kaggle/input/extended-dataset/Expanded_dataset/annotations'
    images_path = '/kaggle/input/extended-dataset/Expanded_dataset/images'
    output_annotations_path = '/kaggle/working/yolo_annotations'
    
    os.makedirs( '/kaggle/working/yolo_dataset/train', exist_ok=True)
    os.makedirs( '/kaggle/working/yolo_dataset/val', exist_ok=True)
    shutil.rmtree('/kaggle/working/yolo_dataset/train')
    shutil.rmtree('/kaggle/working/yolo_dataset/val')

    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/train','images'), exist_ok=True)
    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/val','images'), exist_ok=True)
    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/train','labels'), exist_ok=True)
    os.makedirs( os.path.join('/kaggle/working/yolo_dataset/val','labels'), exist_ok=True)
    
    for annotation_file in os.listdir(annotations_path):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_path, image_file)

        if not os.path.exists(image_path):
            continue

        img_width, img_height = get_image_dimensions(image_path)

        with open(os.path.join(annotations_path, annotation_file), 'r') as file:
            lines = file.readlines()
            annotations = [line.strip().split(',') for line in lines]
            convert_to_yolo_format(annotation_file, annotations, img_width, img_height, given_,output_annotations_path)


    # Split the data
    image_files = os.listdir(images_path)
    train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

    # Move files
    for file in train_files:
        shutil.copy(os.path.join(images_path, file), '/kaggle/working/yolo_dataset/train/images/')
        shutil.copy(os.path.join(output_annotations_path, file.replace('.jpg', '.txt')), '/kaggle/working/yolo_dataset/train/labels/')

    for file in val_files:
        shutil.copy(os.path.join(images_path, file), '/kaggle/working/yolo_dataset/val/images/')
        shutil.copy(os.path.join(output_annotations_path, file.replace('.jpg', '.txt')), '/kaggle/working/yolo_dataset/val/labels/')


## for test from visdrone

In [4]:
def create_files_test(given_):
    # Paths to the dataset
    annotations_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-val/VisDrone2019-DET-val/annotations'
    images_path = '/kaggle/input/visdrone-dataset/VisDrone2019-DET-val/VisDrone2019-DET-val/images'
    os.makedirs( '/kaggle/working/yolo_dataset/test', exist_ok=True)
    shutil.rmtree('/kaggle/working/yolo_dataset/test')

        
    output_dataset_path = '/kaggle/working/yolo_dataset'
    output_annotations_path = os.path.join(output_dataset_path, 'test/labels')
    output_images_path = os.path.join(output_dataset_path, 'test/images')

    os.makedirs(output_annotations_path, exist_ok=True)
    os.makedirs(output_images_path, exist_ok=True)

    
    # Process all annotation files
    for annotation_file in os.listdir(annotations_path):
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(images_path, image_file)

        if not os.path.exists(image_path):
            continue

        img_width, img_height = get_image_dimensions(image_path)

        with open(os.path.join(annotations_path, annotation_file), 'r') as file:
            lines = file.readlines()
            annotations = [line.strip().split(',') for line in lines]
            convert_to_yolo_format(annotation_file, annotations, img_width, img_height, given_,output_annotations_path)

            # Copy the image to the new dataset location
            output_image_path = os.path.join(output_images_path, image_file)
            if not os.path.exists(output_image_path):
                Image.open(image_path).save(output_image_path)

In [5]:
def classes_label(class_name):
    create_files_train_val(class_name)
    create_files_test(class_name)


# 3. Import yolov8

In [6]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")

from ultralytics import YOLO

# Load a YOLOv8 model
model = YOLO('/kaggle/working/yolov8n.pt')  # You can also use yolov8m.pt, yolov8l.pt, etc.

100%|██████████| 6.23M/6.23M [00:00<00:00, 21.1MB/s]


In [7]:
classes_label(0)

In [8]:
data_config = """
train: /kaggle/working/yolo_dataset/train/images
val: /kaggle/working/yolo_dataset/val/images
test: /kaggle/working/yolo_dataset/test/images

nc: 1  # number of classes 
names: ['pedestrian']  # class names
"""

with open('/kaggle/working/yolo_dataset/data.yaml', 'w') as file:
    file.write(data_config)

# 4. Train Class wise (only Class 0 - pedestrian )

# 15 epoch

In [9]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")


# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=15, imgsz=640, batch=16, name='pedestrian_15')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/yolov8n.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=pedestrian_15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fals

100%|██████████| 755k/755k [00:00<00:00, 3.98MB/s]
2024-06-27 16:50:59,391	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-27 16:51:00,990	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/yolo_dataset/train/labels... 1040 images, 286 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<00:00, 1314.54it/s]


train: New cache created: /kaggle/working/yolo_dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      15/15      3.55G      1.206     0.6448     0.8106        195        640: 100%|██████████| 65/65 [00:12<00:00,  5.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  6.42it/s]

                   all        261       2457      0.726      0.512      0.587      0.346



15 epochs completed in 0.066 hours.
Optimizer stripped from runs/detect/pedestrian_15/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/pedestrian_15/weights/best.pt, 6.2MB

Validating runs/detect/pedestrian_15/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:06<00:00,  1.41it/s]


                   all        261       2457      0.726      0.512      0.587      0.345
Speed: 0.1ms preprocess, 1.7ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/pedestrian_15


In [10]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 86 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:05<00:00,  2.85it/s]


                   all        261       2457      0.733      0.517      0.591       0.35
Speed: 0.1ms preprocess, 15.2ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/pedestrian_152


In [11]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/pedestrian_15/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels... 548 images, 28 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<00:00, 1273.06it/s]

val: New cache created: /kaggle/working/yolo_dataset/test/labels.cache



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:05<00:00,  5.89it/s]


                   all        548       8844     0.0396       0.03      0.021    0.00728
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/val


# 25 epoch

In [12]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")


# Load a YOLOv8 model
model = YOLO('/kaggle/working/runs/detect/pedestrian_15/weights/last.pt')  # You can also use yolov8m.pt, yolov8l.pt, etc.

# Train the model 10prev + 15 new = 25
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=10, imgsz=640, batch=16, name='pedestrian_25')

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/pedestrian_15/weights/last.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=pedestrian_25, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, 

train: Scanning /kaggle/working/yolo_dataset/train/labels.cache... 1040 images, 286 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 86 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]


...

      10/10      3.12G      1.119     0.5742      0.811         52        640: 100%|██████████| 65/65 [00:12<00:00,  5.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  5.36it/s]

                   all        261       2457      0.729      0.542      0.616      0.374



10 epochs completed in 0.049 hours.
Optimizer stripped from runs/detect/pedestrian_25/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/pedestrian_25/weights/best.pt, 6.2MB

Validating runs/detect/pedestrian_25/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:06<00:00,  1.40it/s]


                   all        261       2457      0.733      0.541      0.617      0.374
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 5.1ms postprocess per image
Results saved to runs/detect/pedestrian_25


In [13]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 86 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:05<00:00,  2.97it/s]


                   all        261       2457      0.737      0.542      0.618      0.378
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/pedestrian_252


In [14]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/pedestrian_25/weights/best.pt')

# Define the new data.yaml file path
new_data_path = '/kaggle/working/yolo_dataset/data.yaml'

# Evaluate the model on the new dataset
test_results = model.val(data=new_data_path, split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels.cache... 548 images, 28 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:05<00:00,  5.99it/s]


                   all        548       8844     0.0461     0.0163     0.0235    0.00842
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val2


# 50 epoch

In [15]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")

model = YOLO('/kaggle/working/runs/detect/pedestrian_25/weights/last.pt')

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=50, imgsz=640, batch=16, name='pedestrian_50')



/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/pedestrian_25/weights/last.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=pedestrian_50, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, 

train: Scanning /kaggle/working/yolo_dataset/train/labels.cache... 1040 images, 286 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 86 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]


...

      50/50      3.24G     0.9556     0.4564     0.7852         47        640: 100%|██████████| 65/65 [00:12<00:00,  5.25it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  6.55it/s]

                   all        261       2457      0.811       0.58       0.69      0.451



50 epochs completed in 0.234 hours.
Optimizer stripped from runs/detect/pedestrian_50/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/pedestrian_50/weights/best.pt, 6.2MB

Validating runs/detect/pedestrian_50/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:06<00:00,  1.36it/s]


                   all        261       2457      0.809      0.574      0.689      0.455
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/pedestrian_50


In [16]:

val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')



Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 86 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:05<00:00,  2.92it/s]


                   all        261       2457      0.812      0.578      0.693      0.465
Speed: 0.2ms preprocess, 2.4ms inference, 0.0ms loss, 5.1ms postprocess per image
Results saved to runs/detect/pedestrian_502


In [17]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/pedestrian_50/weights/best.pt')

# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels.cache... 548 images, 28 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:05<00:00,  6.35it/s]


                   all        548       8844     0.0323    0.00893     0.0164    0.00617
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val3


# 100 eopch

In [18]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")

model = YOLO('/kaggle/working/runs/detect/pedestrian_50/weights/last.pt')

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=50, imgsz=640, batch=16, name='pedestrian_100')




/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/pedestrian_50/weights/last.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=pedestrian_100, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False,

train: Scanning /kaggle/working/yolo_dataset/train/labels.cache... 1040 images, 286 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 86 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]


...

      50/50      3.25G     0.9068     0.4269     0.7811         47        640: 100%|██████████| 65/65 [00:12<00:00,  5.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  6.45it/s]

                   all        261       2457      0.836      0.598      0.717      0.476



50 epochs completed in 0.234 hours.
Optimizer stripped from runs/detect/pedestrian_100/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/pedestrian_100/weights/best.pt, 6.2MB

Validating runs/detect/pedestrian_100/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:06<00:00,  1.39it/s]


                   all        261       2457      0.851      0.579      0.711      0.481
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/pedestrian_100


In [19]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')



Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 86 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:05<00:00,  3.01it/s]


                   all        261       2457      0.853      0.582      0.716      0.491
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 4.4ms postprocess per image
Results saved to runs/detect/pedestrian_1002


In [20]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/pedestrian_100/weights/best.pt')

# Define the new data.yaml file path
new_data_path = '/kaggle/working/yolo_dataset/data.yaml'

# Evaluate the model on the new dataset
test_results = model.val(data=new_data_path, split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels.cache... 548 images, 28 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:05<00:00,  6.39it/s]


                   all        548       8844     0.0249    0.00565     0.0131    0.00558
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val4


# 200 epoch

In [21]:
import wandb
!export WANDB_MODE=disabled
wandb.init(mode="disabled")

model = YOLO('/kaggle/working/runs/detect/pedestrian_100/weights/last.pt')

# Train the model
results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=200, imgsz=640, batch=16, name='pedestrian_200')


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/pedestrian_100/weights/last.pt, data=/kaggle/working/yolo_dataset/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=pedestrian_200, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fals

train: Scanning /kaggle/working/yolo_dataset/train/labels.cache... 1040 images, 286 backgrounds, 0 corrupt: 100%|██████████| 1040/1040 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 86 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]


...

    200/200       3.6G     0.8407     0.3831     0.7684         56        640: 100%|██████████| 65/65 [00:12<00:00,  5.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  6.09it/s]

                   all        261       2457      0.861      0.616      0.751      0.512



200 epochs completed in 0.872 hours.
Optimizer stripped from runs/detect/pedestrian_200/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/pedestrian_200/weights/best.pt, 6.3MB

Validating runs/detect/pedestrian_200/weights/best.pt...
Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:06<00:00,  1.42it/s]


                   all        261       2457      0.871      0.603      0.753      0.526
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/pedestrian_200


In [22]:
val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')


Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/val/labels.cache... 261 images, 86 backgrounds, 0 corrupt: 100%|██████████| 261/261 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:05<00:00,  2.92it/s]


                   all        261       2457      0.877      0.607      0.755      0.535
Speed: 0.3ms preprocess, 2.0ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/pedestrian_2002


In [23]:
# Load the best model
model = YOLO('/kaggle/working/runs/detect/pedestrian_200/weights/best.pt')


# Evaluate the model on the new dataset
test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')

Ultralytics YOLOv8.2.45 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /kaggle/working/yolo_dataset/test/labels.cache... 548 images, 28 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:05<00:00,  6.63it/s]


                   all        548       8844     0.0155    0.00158    0.00786    0.00413
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/val5


# 500 epoch

In [24]:
# import wandb
# !export WANDB_MODE=disabled
# wandb.init(mode="disabled")


# model = YOLO('/kaggle/working/runs/detect/pedestrian_200/weights/last.pt')

# # Train the model
# results = model.train(data='/kaggle/working/yolo_dataset/data.yaml', epochs=500, imgsz=640, batch=16, name='pedestrian_500')




In [25]:
# val_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='val')



In [26]:
# # Load the best model
# model = YOLO('/kaggle/working/runs/detect/pedestrian_500/weights/best.pt')

# # Evaluate the model on the new dataset
# test_results = model.val(data='/kaggle/working/yolo_dataset/data.yaml', split='test')